In [1]:
!pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 90.2 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 98.0 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 1.0.0rc2
    Uninstalling huggingface-hub-1.0.0rc2:
      Successfully uninstalled huggingface-hub-1.0.0rc2
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.2
    Uninstalling tokenizers-0.21.2:
      Successfully uninstalled tokenizers-0.21.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.53.3
    Uninstalling transformers-4.53.3:
      Successfully uninstalled transformers-4.53.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is t

## Local Inference on GPU 
Model page: https://huggingface.co/openlm-research/open_llama_7b_v2

⚠️ If the generated code snippets do not work, please open an issue on either the [model repo](https://huggingface.co/openlm-research/open_llama_7b_v2)
			and/or on [huggingface.js](https://github.com/huggingface/huggingface.js/blob/main/packages/tasks/src/model-libraries-snippets.ts) 🙏

In [2]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation", model="openlm-research/open_llama_7b_v2")

2025-11-02 04:16:36.339469: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762056996.511071      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762056996.557220      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

pytorch_model.bin.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/593 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/512k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/330 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggin

In [3]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("openlm-research/open_llama_7b_v2")
model = AutoModelForCausalLM.from_pretrained("openlm-research/open_llama_7b_v2")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
import torch
from transformers import LlamaTokenizer, LlamaForCausalLM


## v2 models
model_path = 'openlm-research/open_llama_7b_v2'


tokenizer = LlamaTokenizer.from_pretrained(model_path)
model = LlamaForCausalLM.from_pretrained(
    model_path, torch_dtype=torch.float16, device_map='auto',
)

prompt = 'Q: What is the largest animal?\nA:'
input_ids = tokenizer(prompt, return_tensors="pt").input_ids

generation_output = model.generate(
    input_ids=input_ids, max_new_tokens=32
)
print(tokenizer.decode(generation_output[0]))


`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/transformers/generation/utils.py:2532: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


<s>Q: What is the largest animal?
A: A whale.
Q: What is the smallest animal?
A: A flea.
Q: What is the most dangerous animal?
A


In [11]:
import re
from transformers import GenerationConfig

triples = [
    "Kismet|directed_by|William Dieterle",
    "Kismet|written_by|Edward Knoblock",
    "Kismet|starred_actors|Marlene Dietrich",
    "Flags of Our Fathers|directed_by|Clint Eastwood",
    "Flags of Our Fathers|written_by|Paul Haggis",
    "Flags of Our Fathers|has_genre|War",
    "The Dark Horse|directed_by|Alfred E. Green",
    "The Dark Horse|starred_actors|Bette Davis",
    "The Dark Horse|release_year|1932"
]

prompt_template = """Convert the triple below into a natural English sentence.

Example:
Input: Casablanca|directed_by|Michael Curtiz
Output: Michael Curtiz directed Casablanca.

Now verbalize this triple:
{0}
Output:"""

def verbalize_one(triple):
    prompt = prompt_template.format(triple)
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)
    output_ids = model.generate(
        input_ids,
        max_new_tokens=64,
        temperature=0.4,
        top_p=0.9,
        do_sample=False
    )
    text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    # Strip the prompt portion
    text = text.replace(prompt, "").strip()
    # Take first sentence
    text = re.split(r"[\n\r]", text)[0].strip()
    # Ensure punctuation
    if not text.endswith(('.', '!', '?')):
        text += '.'
    return text

for t in triples:
    print(verbalize_one(t))


William Dieterle directed Kismet.
Edward Knoblock wrote Kismet.
Marlene Dietrich starred in Kismet.
Clint Eastwood directed Flags of Our Fathers.
Paul Haggis wrote Flags of Our Fathers.
War is the genre of Flags of Our Fathers.
Alfred E. Green directed The Dark Horse.
Bette Davis starred in The Dark Horse.
The Dark Horse was released in 1932.


In [40]:
import re
from transformers import GenerationConfig

triples = [
    "Kismet|directed_by|William Dieterle",
    "Kismet|written_by|Edward Knoblock",
    "Kismet|starred_actors|Marlene Dietrich",
    "Flags of Our Fathers|directed_by|Clint Eastwood",
    "Flags of Our Fathers|written_by|Paul Haggis",
    "Flags of Our Fathers|has_genre|War",
    "The Dark Horse|directed_by|Alfred E. Green",
    "The Dark Horse|starred_actors|Bette Davis",
    "The Dark Horse|release_year|1932"
]

prompt_template = """Convert the triple below into a natural English sentence.

Example:
Input: Casablanca|directed_by|Michael Curtiz
Output:Casablanca is directed by Michael Curtiz.

Now verbalize this triple:
{0}
Output:"""

def verbalize_one(triple):
    prompt = prompt_template.format(triple)
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)
    output_ids = model.generate(
        input_ids,
        max_new_tokens=64,
        temperature=0.4,
        top_p=0.9,
        do_sample=False
    )
    text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    # Strip the prompt portion
    text = text.replace(prompt, "").strip()
    # Take first sentence
    text = re.split(r"[\n\r]", text)[0].strip()
    # Ensure punctuation
    if not text.endswith(('.', '!', '?')):
        text += '.'
    return text

for t in triples:
    print(verbalize_one(t))


Kismet is directed by William Dieterle.
Kismet is written by Edward Knoblock.
Marlene Dietrich starred in Kismet.
Flags of Our Fathers is directed by Clint Eastwood.
Flags of Our Fathers is written by Paul Haggis.
Flags of Our Fathers is a war movie.
The Dark Horse is directed by Alfred E. Green.
Bette Davis starred in The Dark Horse.
The Dark Horse was released in 1932.
